In [1]:
import pandas as pd
from metar import Metar
import numpy as np
from IOfuncs import *
import datetime as dt

In [50]:
def make_ml_DataFrame(taf_time, station, lat, lon, metar_path, glamp_path, hrrr_path, delay_hours = 2):
    
    
    metarDF = pd.DataFrame()
    glampDF = pd.DataFrame()
    hrrrDF = pd.DataFrame()
    
    #right now hardcoded to look at metars for 2 hours before TAF due and then predictions after
    for time in range(-6, -delay_hours, 1):
        metar_at_time = get_metar_at_time(taf_time + dt.timedelta(hours = time), metar_path).T
        metarDF[f'metar {time}'] = metar_at_time
    
    work_time = dt.timedelta(hours=-delay_hours)
    glamp_data = get_glamp_at_time(taf_time + work_time, glamp_path, station, download=True)
    hrrr_data = get_hrrr_at_time(taf_time + work_time, hrrr_path, lat, lon, download=True)
    glamp_synoptic_offset = (taf_time.hour - delay_hours) % 6 - 1
    for time in range(-delay_hours, 7, 1):
        glampDF[f'glamp {time}'] = glamp_data.iloc[time + delay_hours + glamp_synoptic_offset]
        hrrrDF[f'hrrr {time}'] = hrrr_data.iloc[time + delay_hours]
    
    df = pd.concat([metarDF, glampDF, hrrrDF])
    v = df.unstack().to_frame().sort_index(level=1).T
    v.columns = v.columns.map('_'.join)
    return v.dropna(axis = 1)

In [51]:
taf_time = dt.datetime(year = 2021, month = 8, day = 21, hour = 18, minute = 16)

In [52]:
make_ml_DataFrame(taf_time, 'kbos', 42.3656, -71.0096, 'Data/BOS.csv', 'Data/GLAMP data/', 'Data/hrrr/')

,hrrr -1_DPT_1000mb,hrrr -2_DPT_1000mb,hrrr 0_DPT_1000mb,hrrr 1_DPT_1000mb,hrrr 2_DPT_1000mb,hrrr 3_DPT_1000mb,hrrr 4_DPT_1000mb,hrrr 5_DPT_1000mb,hrrr 6_DPT_1000mb,hrrr -1_DPT_2m_above_ground,...,glamp 6_wdr,glamp -1_wsp,glamp -2_wsp,glamp 0_wsp,glamp 1_wsp,glamp 2_wsp,glamp 3_wsp,glamp 4_wsp,glamp 5_wsp,glamp 6_wsp
0,292.8,293.5,292.2,292.0,291.8,294.0,293.5,292.2,293.2,293.2,...,90.0,8.0,7.0,9.0,9.0,9.0,9.0,8.0,8.0,7.0


In [79]:
def make_ml_training_DataFrame(taf_time, station, lat, lon, metar_path, glamp_path, hrrr_path, delay_hours = 2):
    df = make_ml_DataFrame(taf_time, station, lat, lon, metar_path, glamp_path, hrrr_path, delay_hours = delay_hours)
    metar_at_time = get_metar_at_time(taf_time, metar_path)
    vis = metar_at_time['vsby'][0]
    ceiling = find_ceiling_height(metar_at_time['metar'].iloc[0])
    if ceiling is None:
        ceiling = 100000
        
    if ceiling < 500 or vis < 1:
        conditions = 0
    elif ceiling < 1000 or vis < 3:
        conditions = 1
    elif ceiling < 3000 or vis < 5:
        conditions = 2
    else:
        conditions = 3
    
    df['flight category'] = conditions
    
    return df

In [80]:
make_ml_training_DataFrame(taf_time, 'kbos', 42.3656, -71.0096, 'Data/BOS.csv', 'Data/GLAMP data/', 'Data/hrrr/')

KeyError: 0